# Pancreas Segmentation
2D-UNET approach.
Data are supposed to be on Google Drive

## Test Setup
- Image resolution: 128x128x128
- Network architecture: 8-16-32-64-32-16-8
- Constant Learning Rate


## Environment Setup
Here we setup the environment. This is a required step to get access to data from Google Drive.

In [ ]:
# SETUP FOR DRIVE ENVIRONMENT
from google.colab import drive
ROOT_PATH = '/content/gdrive'
drive.mount(ROOT_PATH, force_remount=True)

Mounted at /content/gdrive


## Python Modules

In [ ]:
# MODULES TO BE IMPORTED
from datetime import datetime
from matplotlib import pyplot as plt
import math
import nibabel as nib
import numpy as np
import os
import pandas as pd
import pickle
import re
import seaborn as sns
from sklearn.utils import shuffle
import sys

import tensorflow as tf
from tensorflow import keras
import time

### Constants
Here you can change the path where data are loaded from, and where the model is saved. You should change the variable DATA_PATH.


In [ ]:
# FOLDER TO LOAD DATA FROM
MAIN_PATH = os.path.join(ROOT_PATH, 'My Drive', 'Colab Notebooks')
CODE_PATH = os.path.join(MAIN_PATH, 'PANCREAS_2.5D', 'Notebooks')
DATA_PATH = os.path.join(MAIN_PATH, 'PANCREAS_2.5D', 'Data', 'Training')
MODEL_PATH = os.path.join(MAIN_PATH, 'PANCREAS_2.5D', 'Model')

# SETUP
# Volume sizeTest
N_ROWS = 128
N_COLUMNS = 128
N_SLICES = 128

# Type
SHAPE_OF_INTEREST = 'pancreas'
VOLUME_TYPE = 'nii'  # double
VOLUME_TEMPLATE = ("{path}/Patient{patient}/volumeCT_reshape_{shape}_"
                   "{rows}_{columns}_{slices}_{patient}.{volume}")

LABEL_TEMPLATE = ("{path}/Patient{patient}/volumeLabel_reshape_{shape}_"
                  "{rows}_{columns}_{slices}_{patient}.{volume}")
########################
#     SEGMENTATION     #
########################
N_CLASSES = 3
'''
C0: background      (value 0 in the original label volume)
C1: 'pancreas'      (value 1 in the original label volume)
C2: pancreas lesion (value 2 in the original label volume)
'''

########################
#         DATA         #
########################
# TRAINING PERCENTAGE
TRAINING_PERC_CASES = 0.80
# VALIDATION
VALIDATION_PERC_CASES = 0.10
# PERCENTAGE
TEST_PERC_CASES = 1 - TRAINING_PERC_CASES - VALIDATION_PERC_CASES

########################
#         MODEL        #
########################
# Concatenation
CONCATENATION_DIRECTION_OF_FEATURES = 3
# OPTIMIZER
# Regularization factor lambda
L2_REG_LAMBDA = 0.001
# Maximum nuber of epochs
MAX_EPOCHS = 200
'''
TAU factor in the learning rate function
(INITIAL_LEARNING_RATE - FINAL_LEARNING_RATE) * 1 / (1 + math.exp((epoch_number - MAX_EPOCHS) / TAU_EPOCHS))) + FINAL_LEARNING_RATE)
'''
TAU_EPOCHS = 25
# Size for batch normalization
BATCH_SIZE = 10

## Split dataset in training, validation, and test
Here we split the dataset file paths into train, validation,and test sets according to the specified percentages. This is required so that the two generators can get the correct data.

In [ ]:
volumes_list = []  # List of volume file names
labels_list = []  # List of label file names

total_number_of_cases = 0
# For loop over all the folders in data path
for patient_index, patient_folder in enumerate(os.listdir(DATA_PATH)):
    case_id = "{:0>3}".format(patient_index + 1)
    # Set up volume and label path
    volume_path = VOLUME_TEMPLATE.format(
        path=DATA_PATH,
        patient=case_id,
        shape=SHAPE_OF_INTEREST,
        rows=N_ROWS,
        columns=N_COLUMNS,
        slices=N_SLICES,
        volume=VOLUME_TYPE
    )
    label_path = LABEL_TEMPLATE.format(
        path=DATA_PATH,
        patient=case_id,
        shape=SHAPE_OF_INTEREST,
        rows=N_ROWS,
        columns=N_COLUMNS,
        slices=N_SLICES,
        volume=VOLUME_TYPE
    )
    # Check if both paths exist
    if os.path.exists(volume_path) and os.path.exists(label_path):
        total_number_of_cases += 1
        volumes_list.append(volume_path)
        labels_list.append(label_path)

print('Total number of cases: {}'.format(total_number_of_cases))

# Get train and test set
# Get percentages for splitting
training_index = int(total_number_of_cases * TRAINING_PERC_CASES)
validation_index = training_index + \
    int(total_number_of_cases * VALIDATION_PERC_CASES)
test_index = validation_index + \
    int(total_number_of_cases*(1-TRAINING_PERC_CASES-VALIDATION_PERC_CASES))

# Split in train, validation, and test
train_volumes_list = volumes_list[0:training_index]
train_labels_list = labels_list[0:training_index]
validation_volumes_list = volumes_list[training_index:validation_index]
validation_labels_list = labels_list[training_index:validation_index]
test_volumes_list = volumes_list[validation_index:test_index]
test_labels_list = labels_list[validation_index:test_index]

print('Number of training cases: {}'.format(len(train_volumes_list)))
print('Number of validation cases: {}'.format(len(validation_volumes_list)))

## Data generator
This is the generator that loads data for the model. The generator is used to load only the required samples for the current batch of the network, thus reducing the total RAM required.

### Class for DataGenerator
This is the class used for the data generator.

In [ ]:
class DataGenerator(keras.utils.Sequence):
    '''
    Class used for data generators.
    Reference:
    '''

    def __init__(self, id_list, batch_size=10, dim=(200, 200), n_slices=128, shuffle=True):
        '''
        Function called when initializing the class.
        '''
        self.id_list = id_list
        for patient_counter, patient_index in enumerate(self.id_list):
            for slice_number in range(n_slices):
                if (slice_number == 0):
                    patient_list = np.array([[patient_index, slice_number]])
                else:
                    patient_list = np.append(
                        patient_list, [[patient_index, slice_number]], axis=0)
            if (patient_counter == 0):
                self.slices_list = patient_list
            else:
                self.slices_list = np.append(
                    self.slices_list, patient_list, axis=0)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.n_slices = n_slices
        self.dim = dim
        self.on_epoch_end()

    def on_epoch_end(self):
        '''
        Updates indexes after each epoch. If shuffle is set
        to True, the indexes are shuffled. Shuffling the order in which examples
        are fed to the classifier is helpful so that batches between
        epochs do not look alike. Doing so will eventually make our model more robust.
        '''
        self.indexes = np.arange(len(self.slices_list))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        '''
        Generates data containing batch_size samples
        X : (n_samples, *dim, n_channels)
        '''
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        Y = np.empty((self.batch_size, *self.dim))

        # Generate data
        for index, ID in enumerate(list_IDs_temp):
            # Store volume
            temp_volume = nib.load(ID[0][0])
            temp_volume = temp_volume.get_fdata()
            temp_volume = np.asarray(temp_volume)
            X[index, :, :] = temp_volume[:, :, ID[1]]
            # Store label
            temp_label = nib.load(ID[0][1])
            temp_label = temp_label.get_fdata()
            temp_label = np.asarray(temp_label)
            Y[index, :, :] = temp_label[:, :, ID[1]]

        X = X.reshape(X.shape + (1,))  # necessary to give it as input to model
        Y = self.remap_labels(Y)
        return X, Y

    # Map 3D to 4D labels
    def remap_labels(self, labels_3D):
        labels_4D = np.zeros(labels_3D.shape + (N_CLASSES, ))
        # Scan the classes
        for c in range(N_CLASSES):
            temp_indexes = np.where(labels_3D == c)
            labels_4D[temp_indexes +
                      (np.ones(temp_indexes[0].shape, dtype='int') * c, )] = 1
        return labels_4D

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.slices_list) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        id_list_temp = [self.slices_list[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(id_list_temp)

        return X, y

## Custom Metrics

In [ ]:
def IoUMetricFunction(y_true, y_pred, class_value):
    '''
    Compute metric IoU for parameter y_true and y_pred only for the
    specified class.

    Input y_true and y_pred is supposed to be 5-dimensional:
    (batch, x, y, softmax_probabilities)
    '''
    class_IoU_list = []
    y_true = tf.cast(y_true, 'bool')
    # argmax to choose which class the model predicts for each voxel
    y_pred = tf.argmax(y_pred, axis=-1, output_type='int64')
    y_pred = tf.one_hot(indices=y_pred, depth=N_CLASSES,
                        axis=-1, dtype='int64')
    y_pred = tf.cast(y_pred, 'bool')
    y_true_c = y_true[:, :, :, class_value]
    y_pred_c = y_pred[:, :, :, class_value]
    tp = tf.math.count_nonzero(tf.logical_and(y_true_c, y_pred_c))
    fn = tf.math.count_nonzero(tf.logical_and(
        tf.math.logical_xor(y_true_c, y_pred_c), y_true_c))
    fp = tf.math.count_nonzero(tf.logical_and(
        tf.math.logical_xor(y_true_c, y_pred_c), y_pred_c))
    # single scalar, already averaged over different instances
    return tp / (tp + fn + fp)


def IouMetricFactory(class_value):
    '''
    This function is only used to assign a name to the given IoU metric.
    '''
    def fn(y_true, y_pred):
        return IoUMetricFunction(y_true, y_pred, class_value)

    fn.__name__ = 'class_{}_IoU'.format(class_value)
    return fn


# Create a list of functions, so that they can be used in model training.
my_metrics = []
for c in range(0, N_CLASSES):
    my_metrics.append(IouMetricFactory(c))

print(my_metrics)

[<function IouMetricFactory.<locals>.fn at 0x7f2abcce8158>, <function IouMetricFactory.<locals>.fn at 0x7f2abccccea0>, <function IouMetricFactory.<locals>.fn at 0x7f2abccccf28>]


## Model Setup
Here, we setup the model. We define the model with custom callbacks, architecture, and so on.

### Model callbacks
- **early stopping and time monitoring**: this callback checks the training loss, and it its value is lower than 0.001, it stops the training process. It also prints out time information, so that we can find out how longer the training process is.
- **validation early stopping**: this callback checks if, for three consecutive times, the IoU metric on class 1 and class 2 is greater than 95%. If this is true, then the training is stopped.
- **learning rate**: this is a learning rate scheduler that allows to set up a schedule for the learning rate.
-**metrics plot**: this callback plots the accuracy and loss on both the training and validation set in a figure that is stored in the model directory.

In [ ]:
import seaborn as sns
# Early stopping and time monitoring callback


class EarlyStoppingAndInfo(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        global tic
        toc = datetime.now()
        seconds_training = (toc-tic).total_seconds()
        hours, remainder = divmod(seconds_training, 3600)
        minutes, seconds = divmod(remainder, 60)
        print('\n{:02d}:{:02d}:{:02d} spent for training'.format(
            math.floor(hours), math.floor(minutes), math.floor(seconds)))
        if logs['loss'] < 0.001:
            self.model.stop_training = True
            print('\nNOTE: out of training because overfitting')


class ValidationEarlyStopping(tf.keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        # Init variables
        self.epoch_counter = 0    # Epoch counter
        self.val_class_1_IoU = []  # Class 1 IoU
        self.val_class_2_IoU = []  # Class 2 IoU

    def on_epoch_end(self, epoch, logs={}):
        self.val_class_1_IoU.append(logs.get('val_class_1_IoU'))
        self.val_class_2_IoU.append(logs.get('val_class_2_IoU'))
        # Check if last three elements are greater than 0.95 for both classes
        class_1_check = (
            (np.asarray(self.val_class_1_IoU[-3:]) >= 0.95).sum() == 3)
        class_2_check = (
            (np.asarray(self.val_class_2_IoU[-3:]) >= 0.95).sum() == 3)
        if (class_1_check and class_2_check and self.epoch_counter >= 50):
            self.model.stop_training = True
            print('\nNOTE: IoU greater than 95% for 5 consecutive epochs.')


earlyStopperAndInfo = EarlyStoppingAndInfo()

# Learning rate program:


def my_learning_rate_program(epoch_number=0, current_learning_rate=0):
    """ 'current_learning_rate' argument not used, just for the sake of completeness """
    return ((INITIAL_LEARNING_RATE - FINAL_LEARNING_RATE) * 1 / (1 + math.exp((epoch_number - MAX_EPOCHS) / TAU_EPOCHS))) + FINAL_LEARNING_RATE


learningRateScheduler = tf.keras.callbacks.LearningRateScheduler(
    my_learning_rate_program, verbose=1)

# plt.plot(range(0, MAX_EPOCHS), [my_learning_rate_program(epoch_number = n) for n in range(0, MAX_EPOCHS)])
# plt.title('learning rate with epochs:')
# plt.show()

# Plot accuracy and loss


class PlotMetrics(tf.keras.callbacks.Callback):

    def __init__(self, **kwargs):
        self.previous_data = False
        self.epoch_counter = 0
        super()

    def set_folder(self, folder):
        self.folder = folder

    def load_data(self, folder):
        self.previous_data = True
        self.data = pd.read_csv(os.path.join(
            MODEL_PATH, folder, 'TrainingMonitoring.csv'))

    def on_train_begin(self, logs={}):
        if (not self.previous_data):
            # Init all arrays
            self.data = pd.DataFrame(columns=['Tr Loss',
                                              'Cl0 IoU',
                                              'Cl1 IoU',
                                              'Cl2 IoU',
                                              'Val Loss',
                                              'VCl0 IoU',
                                              'VCl1 IoU',
                                              'VCl2 IoU'],
                                     dtype=float)

    def on_epoch_end(self, batch, logs={}):
        # Append metrics
        self.data.loc[self.epoch_counter, 'Tr Loss'] = float(logs.get('loss'))
        self.data.loc[self.epoch_counter, 'Cl0 IoU'] = float(
            logs.get('class_0_IoU'))
        self.data.loc[self.epoch_counter, 'Cl1 IoU'] = float(
            logs.get('class_1_IoU'))
        self.data.loc[self.epoch_counter, 'Cl2 IoU'] = float(
            logs.get('class_2_IoU'))
        self.data.loc[self.epoch_counter, 'Val Loss'] = float(
            logs.get('val_loss'))
        self.data.loc[self.epoch_counter, 'VCl0 IoU'] = float(
            logs.get('val_class_0_IoU'))
        self.data.loc[self.epoch_counter, 'VCl1 IoU'] = float(
            logs.get('val_class_1_IoU'))
        self.data.loc[self.epoch_counter, 'VCl2 IoU'] = float(
            logs.get('val_class_2_IoU'))
        self.epoch_counter += 1

        # print(self.data)
        # .. And plot them!

        plt.figure(figsize=(10, 10))
        sns.set(style='darkgrid')

        ax = sns.lineplot(x=range(0, len(self.data)), y='Tr Loss', data=self.data, ci=None,
                          label='Tr Loss', linewidth=3)
        sns.lineplot(x=range(0, len(self.data)), y='Cl0 IoU', data=self.data, ci=None,
                     label='0 IoU', linewidth=3, ax=ax)
        sns.lineplot(x=range(0, len(self.data)), y='Cl1 IoU', data=self.data, ci=None,
                     label='1 IoU', linewidth=3, ax=ax)
        sns.lineplot(x=range(0, len(self.data)), y='Cl2 IoU', data=self.data, ci=None,
                     label='2 IoU', linewidth=3, ax=ax)
        sns.lineplot(x=range(0, len(self.data)), y='Val Loss', data=self.data, ci=None,
                     label='V Loss', linewidth=3, ax=ax)
        sns.lineplot(x=range(0, len(self.data)), y='VCl0 IoU', data=self.data, ci=None,
                     label='V0 IoU', linewidth=3, ax=ax)
        sns.lineplot(x=range(0, len(self.data)), y='VCl1 IoU', data=self.data, ci=None,
                     label='V1 IoU', linewidth=3, ax=ax)
        sns.lineplot(x=range(0, len(self.data)), y='VCl2 IoU', data=self.data, ci=None,
                     label='V2 IoU', linewidth=3, ax=ax)
        ax.set_xlabel("Epochs", fontsize=16)
        ax.set_ylabel("", fontsize=16)
        ax.set_title('Training Monitoring', fontsize=20)
        # Save Figure
        fig = ax.get_figure()
        fig.savefig(os.path.join(
            MODEL_PATH, self.folder, 'TrainingMonitoring.png'))
        # Save dataframe
        self.data.to_csv(os.path.join(MODEL_PATH, self.folder,
                         'TrainingMonitoring.csv'), index=False)
        plt.close()


plotMetrics = PlotMetrics()

# my_callbacks_list = [earlyStopperAndInfo, learningRateScheduler, plotMetrics]
my_callbacks_list = [earlyStopperAndInfo, plotMetrics]

### Model architecture

In [ ]:
# Network Model

# Initialize input size
my_input_tensor = tf.keras.layers.Input(
    shape=(N_ROWS, N_COLUMNS, 1))  # , dtype = 'float16'

# Here we define some parameters of the networks

# First processing layer
if True:
    # Convolution 16 filters
    cumulative_resulting_tensor = tf.keras.layers.Conv2D(16, kernel_size=(3, 3),
                                                         strides=(1, 1),
                                                         padding='same',
                                                         activation='linear',
                                                         kernel_regularizer=tf.keras.regularizers.l2(
                                                             L2_REG_LAMBDA),
                                                         bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(my_input_tensor)
    # Batch for averaging
    cumulative_resulting_tensor = tf.keras.layers.BatchNormalization(
        axis=-1)(cumulative_resulting_tensor)
    # RELU
    cumulative_resulting_tensor = tf.keras.layers.Activation(
        'relu')(cumulative_resulting_tensor)
    # Convolution 16 filters
    cumulative_resulting_tensor = tf.keras.layers.Conv2D(16, kernel_size=(3, 3),
                                                         strides=(1, 1),
                                                         padding='same',
                                                         activation='linear',
                                                         kernel_regularizer=tf.keras.regularizers.l2(
                                                             L2_REG_LAMBDA),
                                                         bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(cumulative_resulting_tensor)
    # Batch for averaging
    cumulative_resulting_tensor = tf.keras.layers.BatchNormalization(
        axis=-1)(cumulative_resulting_tensor)
    # RELU
    cumulative_resulting_tensor = tf.keras.layers.Activation(
        'relu')(cumulative_resulting_tensor)
    # Temporary output
    intermediate_tensor_1 = cumulative_resulting_tensor
    # Maxpooling
    cumulative_resulting_tensor = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),
                                                               strides=(2, 2),
                                                               padding='same')(cumulative_resulting_tensor)

# Second processing layer
if True:
    # Convolution 32 filters
    cumulative_resulting_tensor = tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                                                         strides=(1, 1),
                                                         padding='same',
                                                         activation='linear',
                                                         kernel_regularizer=tf.keras.regularizers.l2(
                                                             L2_REG_LAMBDA),
                                                         bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(cumulative_resulting_tensor)
    # Batch for averaging
    cumulative_resulting_tensor = tf.keras.layers.BatchNormalization(
        axis=-1)(cumulative_resulting_tensor)
    # RELU
    cumulative_resulting_tensor = tf.keras.layers.Activation(
        'relu')(cumulative_resulting_tensor)
    # Convolution 32 filters
    cumulative_resulting_tensor = tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                                                         strides=(1, 1),
                                                         padding='same',
                                                         activation='linear',
                                                         kernel_regularizer=tf.keras.regularizers.l2(
                                                             L2_REG_LAMBDA),
                                                         bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(cumulative_resulting_tensor)
    # Batch for averaging
    cumulative_resulting_tensor = tf.keras.layers.BatchNormalization(
        axis=-1)(cumulative_resulting_tensor)
    # RELU
    cumulative_resulting_tensor = tf.keras.layers.Activation(
        'relu')(cumulative_resulting_tensor)
    # Temporary output
    intermediate_tensor_2 = cumulative_resulting_tensor
    # Maxpooling
    cumulative_resulting_tensor = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),
                                                               strides=(2, 2), padding='same')(cumulative_resulting_tensor)

# Third processing layer
if True:
    # Convolution 64 filters
    cumulative_resulting_tensor = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                                                         strides=(1, 1),
                                                         padding='same',
                                                         activation='linear',
                                                         kernel_regularizer=tf.keras.regularizers.l2(
                                                             L2_REG_LAMBDA),
                                                         bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(cumulative_resulting_tensor)
    # Batch for averaging
    cumulative_resulting_tensor = tf.keras.layers.BatchNormalization(
        axis=-1)(cumulative_resulting_tensor)
    # RELU
    cumulative_resulting_tensor = tf.keras.layers.Activation(
        'relu')(cumulative_resulting_tensor)
    # Convolution 64 filters
    cumulative_resulting_tensor = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                                                         strides=(1, 1),
                                                         padding='same',
                                                         activation='linear',
                                                         kernel_regularizer=tf.keras.regularizers.l2(
                                                             L2_REG_LAMBDA),
                                                         bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(cumulative_resulting_tensor)
    # Batch for averaging
    cumulative_resulting_tensor = tf.keras.layers.BatchNormalization(
        axis=-1)(cumulative_resulting_tensor)
    # RELU
    cumulative_resulting_tensor = tf.keras.layers.Activation(
        'relu')(cumulative_resulting_tensor)
    # Temporary output
    intermediate_tensor_3 = cumulative_resulting_tensor
    # Maxpooling
    cumulative_resulting_tensor = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),
                                                               strides=(2, 2),
                                                               padding='same')(cumulative_resulting_tensor)
   # Forth processing layer
if True:
    # Convolution 64 filters
    cumulative_resulting_tensor = tf.keras.layers.Conv2D(128, kernel_size=(3, 3),
                                                         strides=(1, 1),
                                                         padding='same',
                                                         activation='linear',
                                                         kernel_regularizer=tf.keras.regularizers.l2(
                                                             L2_REG_LAMBDA),
                                                         bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(cumulative_resulting_tensor)
    # Batch for averaging
    cumulative_resulting_tensor = tf.keras.layers.BatchNormalization(
        axis=-1)(cumulative_resulting_tensor)
    # RELU
    cumulative_resulting_tensor = tf.keras.layers.Activation(
        'relu')(cumulative_resulting_tensor)
    # Convolution 64 filters
    cumulative_resulting_tensor = tf.keras.layers.Conv2D(128, kernel_size=(3, 3),
                                                         strides=(1, 1),
                                                         padding='same',
                                                         activation='linear',
                                                         kernel_regularizer=tf.keras.regularizers.l2(
                                                             L2_REG_LAMBDA),
                                                         bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(cumulative_resulting_tensor)
    # Batch for averaging
    cumulative_resulting_tensor = tf.keras.layers.BatchNormalization(
        axis=-1)(cumulative_resulting_tensor)
    # RELU
    cumulative_resulting_tensor = tf.keras.layers.Activation(
        'relu')(cumulative_resulting_tensor)
    # Temporary output
    intermediate_tensor_4 = cumulative_resulting_tensor
    # Maxpooling
    # cumulative_resulting_tensor = keras.layers.MaxPooling3D(pool_size = (2, 2, 2), strides = (2, 2, 2), padding = 'same')(cumulative_resulting_tensor)

# Fourth processing layer
if True:
    # Deconvolution 32 filters
    cumulative_resulting_tensor = tf.keras.layers.Conv2DTranspose(64, kernel_size=(2, 2),
                                                                  strides=(
                                                                      2, 2),
                                                                  padding='same',
                                                                  activation='linear',
                                                                  kernel_regularizer=tf.keras.regularizers.l2(
                                                                      L2_REG_LAMBDA),
                                                                  bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(cumulative_resulting_tensor)
    # RELU
    cumulative_resulting_tensor = tf.keras.layers.Activation(
        'relu')(cumulative_resulting_tensor)
    # Concatenation
    cumulative_resulting_tensor = tf.keras.layers.Concatenate(
        axis=CONCATENATION_DIRECTION_OF_FEATURES)([intermediate_tensor_3, cumulative_resulting_tensor])
    # Convolution 32 filters
    cumulative_resulting_tensor = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                                                         strides=(1, 1),
                                                         padding='same',
                                                         activation='linear',
                                                         kernel_regularizer=tf.keras.regularizers.l2(
                                                             L2_REG_LAMBDA),
                                                         bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(cumulative_resulting_tensor)
    # RELU
    cumulative_resulting_tensor = tf.keras.layers.Activation(
        'relu')(cumulative_resulting_tensor)
    # Convolution 32 filters
    cumulative_resulting_tensor = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                                                         strides=(1, 1),
                                                         padding='same',
                                                         activation='linear',
                                                         kernel_regularizer=tf.keras.regularizers.l2(
                                                             L2_REG_LAMBDA),
                                                         bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(cumulative_resulting_tensor)
    # RELU
    cumulative_resulting_tensor = tf.keras.layers.Activation(
        'relu')(cumulative_resulting_tensor)


# Fourth processing layer
if True:
    # Deconvolution 32 filters
    cumulative_resulting_tensor = tf.keras.layers.Conv2DTranspose(32, kernel_size=(2, 2),
                                                                  strides=(
                                                                      2, 2),
                                                                  padding='same',
                                                                  activation='linear',
                                                                  kernel_regularizer=tf.keras.regularizers.l2(
                                                                      L2_REG_LAMBDA),
                                                                  bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(cumulative_resulting_tensor)
    # RELU
    cumulative_resulting_tensor = tf.keras.layers.Activation(
        'relu')(cumulative_resulting_tensor)
    # Concatenation
    cumulative_resulting_tensor = tf.keras.layers.Concatenate(
        axis=CONCATENATION_DIRECTION_OF_FEATURES)([intermediate_tensor_2, cumulative_resulting_tensor])
    # Convolution 32 filters
    cumulative_resulting_tensor = tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                                                         strides=(1, 1),
                                                         padding='same',
                                                         activation='linear',
                                                         kernel_regularizer=tf.keras.regularizers.l2(
                                                             L2_REG_LAMBDA),
                                                         bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(cumulative_resulting_tensor)
    # RELU
    cumulative_resulting_tensor = tf.keras.layers.Activation(
        'relu')(cumulative_resulting_tensor)
    # Convolution 32 filters
    cumulative_resulting_tensor = tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                                                         strides=(1, 1),
                                                         padding='same',
                                                         activation='linear',
                                                         kernel_regularizer=tf.keras.regularizers.l2(
                                                             L2_REG_LAMBDA),
                                                         bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(cumulative_resulting_tensor)
    # RELU
    cumulative_resulting_tensor = tf.keras.layers.Activation(
        'relu')(cumulative_resulting_tensor)

# Fifth processing layer
if True:
    # Deconvolution 16 filters
    cumulative_resulting_tensor = tf.keras.layers.Conv2DTranspose(16, kernel_size=(2, 2),
                                                                  strides=(
                                                                      2, 2),
                                                                  padding='same',
                                                                  activation='linear',
                                                                  kernel_regularizer=tf.keras.regularizers.l2(
                                                                      L2_REG_LAMBDA),
                                                                  bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(cumulative_resulting_tensor)
    # RELU
    cumulative_resulting_tensor = tf.keras.layers.Activation(
        'relu')(cumulative_resulting_tensor)
    # Concatenation
    cumulative_resulting_tensor = tf.keras.layers.Concatenate(
        axis=CONCATENATION_DIRECTION_OF_FEATURES)([intermediate_tensor_1, cumulative_resulting_tensor])
    # Convolution 16 filters
    cumulative_resulting_tensor = tf.keras.layers.Conv2D(16, kernel_size=(3, 3),
                                                         strides=(1, 1),
                                                         padding='same',
                                                         activation='linear',
                                                         kernel_regularizer=tf.keras.regularizers.l2(
                                                             L2_REG_LAMBDA),
                                                         bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(cumulative_resulting_tensor)
    # RELU
    cumulative_resulting_tensor = tf.keras.layers.Activation(
        'relu')(cumulative_resulting_tensor)
    # Convolution 16 filters
    cumulative_resulting_tensor = tf.keras.layers.Conv2D(4, kernel_size=(3, 3),
                                                         strides=(1, 1),
                                                         padding='same',
                                                         activation='linear',
                                                         kernel_regularizer=tf.keras.regularizers.l2(
                                                             L2_REG_LAMBDA),
                                                         bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(cumulative_resulting_tensor)
    # RELU
    cumulative_resulting_tensor = tf.keras.layers.Activation(
        'relu')(cumulative_resulting_tensor)

# Sixth processing layer
if True:
    # Convolution 1 filter sigmoidal (to make size converge to final one)
    cumulative_resulting_tensor = tf.keras.layers.Conv2D(N_CLASSES, kernel_size=(1, 1),
                                                         strides=(1, 1),
                                                         padding='same',
                                                         activation='linear',
                                                         kernel_regularizer=tf.keras.regularizers.l2(
                                                             L2_REG_LAMBDA),
                                                         bias_regularizer=tf.keras.regularizers.l2(L2_REG_LAMBDA))(cumulative_resulting_tensor)

    my_output_tensor = tf.keras.layers.Softmax(
        axis=-1)(cumulative_resulting_tensor)

my_model = tf.keras.Model(inputs=[my_input_tensor], outputs=[my_output_tensor])
print('Parameters in model: {}'.format(my_model.count_params()))

Parameters in model: 481923


## Model training
Since it may happen that the training process of our model stops (due to disconnection from Google Colab VM or other problems) we first check if we need to recover the training process. This is done by checking the existance of a file called "Running" that is deleted once the training is completed. If such a file exists, the model recovers the training process, otherwise a new training is started.

---

First, we define some useful functions that we will use to recover previous weights.

In [ ]:
def find_latest_data_folder(path):
    '''
    This function finds the latest data folder among all the data folders.
    It does so by looking at the folder name, that contains information
    about year, month, day and time. By checking the time-delta
    from each folder to the current time, it is possible to find
    the latest one.
    '''
    data_folders = os.listdir(path)  # Get list of data folders
    data_folders_dt = []  # Init datetime list of data folder names
    data_folders_timedelta = []  # Init timedelta list
    for folder in data_folders:
        try:
            # We need two lists, otherwise the indexes won't match
            data_folders_timedelta.append(
                datetime.now() - datetime.strptime(folder, '%Y%m%d_%H%M%S'))
            data_folders_dt.append(folder)
        except ValueError:
            continue
    if (len(data_folders_dt) > 0):
        # Find latest data folder based on timedelta
        latest_data_folder = data_folders_dt[np.argmin(data_folders_timedelta)]
        # Return full path
        data_folder = os.path.join(path, latest_data_folder)
    else:
        data_folder = ''
    return data_folder


def find_latest_weights_file(path):
    '''
    This function finds the latest data folder among all the data folders.
    It does so by looking at the folder name, that contains information
    about year, month, day and time. By checking the time-delta
    from each folder to the current time, it is possible to find
    the latest one.
    '''
    file_names = os.listdir(path)  # Get list of data folders
    highest_epoch_file_name = ''
    max_epoch = -1
    for file_name in file_names:
        if ('weights' in file_name):
            if (file_name[10] == '-'):
                if (int(file_name[8:10]) > max_epoch):
                    max_epoch = int(file_name[8:10])
                    highest_epoch_file_name = file_name
            else:
                if (int(file_name[8:11]) > max_epoch):
                    max_epoch = int(file_name[8:11])
                    highest_epoch_file_name = file_name
    return highest_epoch_file_name, max_epoch

Then, we check if the training of a model started. If so, we recover the weights. Otherwise we start a new training process.

In [ ]:
# Check if we have already started training a model, but the training
# could not be completed
weights_found = False
if (os.path.exists(os.path.join(TEMP_PATH, 'running'))):
    print('Running file found in folder.')
    # Reload weights
    data_folder = find_latest_data_folder(TEMP_PATH)
    if (not (data_folder == '')):
        model_weights_folder = os.path.join(
            data_folder, '{}_{}'.format(N_ROWS, N_COLUMNS))
        weights_file, last_epoch = find_latest_weights_file(
            model_weights_folder)
        if ('weights' in weights_file):
            weights_found = True
            print('Loading weights from: {}'.format(
                os.path.join(model_weights_folder, weights_file)))
            my_model.load_weights(os.path.join(
                model_weights_folder, weights_file))
            plotMetrics.load_data(data_folder)

if (not weights_found):
    print('No previous model found. Starting training from scratch.')
    if (os.path.exists(os.path.join(TEMP_PATH, 'running'))):
        os.remove(os.path.join(TEMP_PATH, 'running'))
    # We need to train the model from scratch
    data_folder = os.path.join(TEMP_PATH,
                               datetime.strftime(datetime.now(), "%Y%m%d_%H%M%S"))
    model_weights_folder = os.path.join(data_folder,
                                        '{}_{}'.format(N_ROWS, N_COLUMNS)
                                        )
    if (not (os.path.exists(model_weights_folder))):
        os.makedirs(model_weights_folder)
    last_epoch = 0

# Set folder where to store training monitoring plot
plotMetrics.set_folder(data_folder)
# Set checkpoint to save weights
model_checkpoint_filepath = os.path.join(model_weights_folder,
                                         'weights.{epoch:03d}-{val_loss:.2f}.hdf5'
                                         )
model_checkpoint_save = tf.keras.callbacks.ModelCheckpoint(filepath=model_checkpoint_filepath,
                                                           save_weights_only=True,
                                                           monitor='val_loss',
                                                           period=5)
my_callbacks_list.append(model_checkpoint_save)
print('Starting model from epoch: {}'.format(last_epoch))

No previous model found. Starting training from scratch.
Starting model from epoch: 0


As a last step, we compile and fit the model. Once the training of the model is complete, we delete the file "Running" and we print out the total time required for the model to complete the training process.

The parameters that can be set in the generator init function and in the fit generator function are:
- **batch_size**: determines the number of samples in each mini batch. Its maximum is the number of all samples, which makes gradient descent accurate, the loss will decrease towards the minimum if the learning rate is small enough, but iterations are slower. Its minimum is 1, resulting in stochastic gradient descent: Fast but the direction of the gradient step is based only on one example, the loss may jump around. batch_size allows to adjust between the two extremes: accurate gradient direction and fast iteration. Also, the maximum value for batch_size may be limited if your model + data set does not fit into the available (GPU) memory.
- **steps_per_epoch**: the number of batch iterations before a training epoch is considered finished. If you have a training set of fixed size you can ignore it but it may be useful if you have a huge data set or if you are generating random data augmentations on the fly, i.e. if your training set has a (generated) infinite size. If you have the time to go through your whole training data set I recommend to skip this parameter.
- **validation_steps**: similar to steps_per_epoch but on the validation data set instead on the training data. If you have the time to go through your whole validation data set I recommend to skip this parameter.

In [ ]:
# Compile model
my_model.compile(
    # optimizer = keras.optimizers.Adam(learning_rate = my_learning_rate_program()), # arg = 0 by default
    optimizer=keras.optimizers.Adam(learning_rate=3e-5),  # arg = 0 by default
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=my_metrics
)

# Get list of file names for volumes and labels
train_array = [list(data)
               for data in zip(train_volumes_list, train_labels_list)]
validation_array = [list(data) for data in zip(
    validation_volumes_list, validation_labels_list)]

# Set up train and validation generator
train_generator = DataGenerator(
    train_array, batch_size=100, dim=(N_ROWS, N_COLUMNS), n_slices=N_SLICES)
validation_generator = DataGenerator(
    validation_array, batch_size=100, dim=(N_ROWS, N_COLUMNS), n_slices=N_SLICES)

# Get current time
tic = datetime.now()

if (not (os.path.exists(os.path.join(TEMP_PATH)))):
    os.makedirs(TEMP_PATH)

# Write file so that we can recover the last weights
if (not (os.path.exists(os.path.join(TEMP_PATH, 'running')))):
    with open(os.path.join(TEMP_PATH, 'running'), 'w') as f:
        f.write('Running')

# Fit model
monitoring = my_model.fit(train_generator,
                          steps_per_epoch=math.ceil(
                              len(train_array)*N_SLICES / BATCH_SIZE),
                          epochs=MAX_EPOCHS,
                          validation_data=validation_generator,
                          validation_steps=math.ceil(
                              len(validation_array)*N_SLICES / 10),
                          callbacks=my_callbacks_list,
                          max_queue_size=1,
                          workers=4,
                          initial_epoch=last_epoch)

# Print out total time
toc = datetime.now()
seconds_training = (toc-tic).total_seconds()
hours, remainder = divmod(seconds_training, 3600)
minutes, seconds = divmod(remainder, 60)
print('\n\nTraining completed!!\n{:02d}:{:02d}:{:02d} spent for training'.format(
    int(hours), int(minutes), int(seconds)))

# Delete file "Running"
if (os.path.exists(os.path.join(TEMP_PATH, 'runnning'))):
    os.remove(os.path.join(TEMP_PATH, 'running'))

# Save the model
owd = os.getcwd()
if (not (os.path.exists(MODEL_PATH))):
    os.makedirs(MODEL_PATH)
os.chdir(MODEL_PATH)
number_of_models = len(os.listdir())
my_model.save('model_{}_{}x{}x{}.h5'.format(
    number_of_models, N_ROWS, N_COLUMNS, N_SLICES))
os.chdir(owd)